In [ ]:
import seisbench.models as sbm

from scripts.evaluate import experiment

In [ ]:
# Default：DeepDenoiserでの実験
model = sbm.DeepDenoiser.from_pretrained("original")

result = experiment(model, "DeepDenoiser")
print(result)